In [1]:
from reflexion4rec.agents import ReactAgent

In [3]:
import os
import openai

openai.api_base = "https://api.closeai-proxy.xyz/v1"

os.environ['OPENAI_API_KEY'] = "sk-wu9r49jGPV2ypCBSOWQL9SgpdeZfKZNQWb8DMhi1T6X7DUl7"  # only for test :)
print(os.environ['OPENAI_API_KEY'])

sk-wu9r49jGPV2ypCBSOWQL9SgpdeZfKZNQWb8DMhi1T6X7DUl7


In [8]:
from reflexion4rec.llms import AnyOpenAILLM

react_llm = AnyOpenAILLM(
    temperature=0,
    max_tokens=100,
    model_name="gpt-3.5-turbo",
    model_kwargs={"stop": "\n"},
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # openai_api_base="https://api.closeai-proxy.xyz/v1"
)

In [5]:
from reflexion4rec.prompts import read_template

react_prompt = read_template("config/prompts/react_prompt.json")['test_react_prompt']

input_variables=['examples', 'input', 'scratchpad'] output_parser=None partial_variables={} template='Solve a rating prediction task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.{examples}\n Input: {input}{scratchpad}' template_format='f-string' validate_template=True


In [16]:
import pandas as pd

df = pd.read_csv("toy.csv")
df['item_attributes'] = np.array(["Title: Lion King, The (1994), Release Date: 01-Jan-1994, Video Release Date: unknown, Genres: Animation|Childrens|Musical"])

data_prompt = read_template("config/prompts/rp.json")["rp_data_prompt"]
test_data = data_prompt.format(
                user_id=df['user_id'][0],
                user_profile=df['user_profile'][0],
                history=df['history'][0],
                target_item_id=df['item_id'][0],
                target_item_attributes=df['item_attributes'][0]
            )
test_data

"[user_id]: user_821\n [user_profile]:\nAge: 37\nGender:male\nOccupation: engineer\n [historical interactions]:\nTitle: Liar Liar (1997), Release Date: 21-Mar-1997, Video Release Date: unknown, Genres: Comedy (rating: 4)\nTitle: Anne Frank Remembered (1995), Release Date: 23-Feb-1996, Video Release Date: unknown, Genres: Documentary (rating: 5)\nTitle: Fargo (1996), Release Date: 14-Feb-1997, Video Release Date: unknown, Genres: Crime|Drama|Thriller (rating: 2)\nTitle: Sense and Sensibility (1995), Release Date: 01-Jan-1995, Video Release Date: unknown, Genres: Drama|Romance (rating: 5)\nTitle: Postino, Il (1994), Release Date: 01-Jan-1994, Video Release Date: unknown, Genres: Drama|Romance (rating: 4)\nTitle: First Wives Club, The (1996), Release Date: 14-Sep-1996, Video Release Date: unknown, Genres: Comedy (rating: 4)\nTitle: Rock, The (1996), Release Date: 07-Jun-1996, Video Release Date: unknown, Genres: Action|Adventure|Thriller (rating: 3)\nTitle: Happy Gilmore (1996), Release D

In [19]:
# test one step

agent = ReactAgent(agent_prompt=react_prompt,
                react_examples="", 
                actor_llm=react_llm)

agent.set_data(input=test_data, context="", gt_answer="1")

print(f'Init: {agent._build_agent_prompt()}')
print("====================================")
agent.step()

Init: Solve a rating prediction task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.
 Input: [user_id]: user_821
 [user_profile]:
Age: 37
Gender:male
Occupation: engineer
 [historical interactions]:
Title: Liar Liar (1997), Release Date: 21-Mar-1997, Video Release Date: unknown, Genres: Comedy (rating: 4)
Title: Anne Frank Remembered (1995), Release Date: 23-Feb-1996, Video Release Date: unknown, Genres: Documentary (rating: 5)
Title: Fargo (1996), Release Date: 14-Feb-1997, Video Release Date: unknown, Genres: Crime|Drama|Thriller (rating: 2)
Title: Sense and Sensibility (1995), Release Date: 01-Jan-1995, Video Release Date: unknown, Genres: Drama|Romance (rating: 5)
Title: Postino, Il (1994), Release Date: 01-Jan-1994, Video Release Date: unknown, Genres: Drama|Romance (rating: 4)
Title: First Wives Club, The (1996), Release Date: 14-Sep-1996, Video Release Date: unknown, Genre